In [1]:
import os
import numpy as np
import torch as tr

from CSWSEM import *
from matplotlib import pyplot as plt

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import torch as tr

from CSWSEM import *
gs_name = 'gs2'

import sys

# sweep params
# nosplit = int(sys.argv[1])
# condition = str(sys.argv[2])
# learn_rate = float(sys.argv[3])
# alfa = float(sys.argv[4])
# lmda = float(sys.argv[5])
# seed = int(sys.argv[6])
stsize = 25

# nb
condition = 'blocked'
stsize = 25
learn_rate = 0.05
seed = 97
alfa = 10
lmda = 1
nosplit = 0


model_tag = 'nosplit_%i__cond_%s__learnrate_%.3f__alfa_%f__lmbda_%f__seed_%i'%(
  int(nosplit),condition,learn_rate,alfa,lmda,seed)
print(model_tag)


# params
exp_kwargs={
    'condition':condition,
    'n_train':160,
    'n_test':40
}



sem_kwargs={
    'nosplit':nosplit,
    'stsize':stsize,
    'lmda':lmda,
    'alfa':alfa,
    'learn_rate':learn_rate,
    'seed':seed,
}

# setup
task = CSWTask()
sem = SEM(**sem_kwargs)

# run
exp,curr = task.generate_experiment(**exp_kwargs)
sem_data = sem.forward_exp(exp)

# record curriculum (not idea, recording with every obs)
sem.data.record_exp('curriculum',curr)

# save
import pandas as pd
save_fpath = 'gsdata/%s/%s'%(gs_name,model_tag)
pd.DataFrame(sem_data).to_csv(save_fpath)



nosplit_0__cond_blocked__learnrate_0.050__alfa_10.000000__lmbda_1.000000__seed_97


In [4]:
L = [pd.DataFrame(sem_data),pd.DataFrame(sem_data)]


In [6]:
pd.concat(L)

,trial,prior,like,active_schema,loss,delta_time,nosplit,stsize,learn_rate,lmda,alfa,seed,curriculum
0,0,"[2.302585092994046, 2.302585092994046]","[-100.98349483682152, -100.98349483682152]",0,0.59315306,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[1.9459101490553132, 2.302585092994046]","[-137.3457983694307, -56.39835630734736]",1,0.47059053,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[1.791759469228055, 1.791759469228055, 2.30258...","[-137.3457983694307, -51.31236764262779, -56.3...",1,0.42183322,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[1.791759469228055, 2.4849066497880004, 2.3025...","[-137.3457983694307, -2.994897901803771, -56.3...",1,0.33222803,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[1.791759469228055, 2.8903717578961645, 2.3025...","[-137.3457983694307, 13.059506009842597, -56.3...",1,0.2823714,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,"[6.386879319362645, 5.683579767338681, 5.64190...","[143.6303254113167, -315.1698993629981, -168.1...",0,0.07204375,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
196,196,"[6.396929655216146, 5.683579767338681, 5.64190...","[147.20164457341684, -315.1698993629981, -168....",0,0.06702559,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
197,197,"[6.406879986069314, 5.683579767338681, 5.64190...","[99.55638947935594, -298.4806790558736, -356.0...",0,0.14086878,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
198,198,"[6.416732282512326, 5.683579767338681, 5.64190...","[-769.3554040555052, -130.13841927508057, 216....",2,0.00037193505,2.113334,0,25,0.05,1,10,97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
